<a href="https://colab.research.google.com/github/nipunagarwala/ee364b_final_project/blob/main/SparseLowRankInv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import sys

import numpy as np
import matplotlib.pyplot as plt
import cvxpy as cp
from numpy import linalg as LA
import random
from scipy.linalg import ldl
import math
import scipy.io as sio
# import fjlt


In [5]:
'''
FIXME: Use sparse representation to save computation
'''

def findSMat(MstarPath, MatlabVarName, AMat=None):
	MStarMat_dict = sio.loadmat(MstarPath, squeeze_me=True)
	# Hardcoded to the 4th entry since that seems to be the name of the key actually having
	# the matrix value
	MStarMat_CSR = MStarMat_dict[MatlabVarName]
	MStarMatDense = np.asarray(MStarMat_CSR.todense())
	Idn = np.identity(MStarMatDense.shape[0])
	MStarAMatProd = np.dot(MStarMatDense, AMat)
	SMat = 2*Idn - MStarAMatProd - MStarAMatProd.T

	return SMat, MStarMatDense



In [6]:
def findThinQ(SMat, EmbedRow, RndType='JLT'):
	SMatOmegaMatProd = np.zeros((EmbedRow, SMat.shape[1]))
	if RndType == 'Gaussian':
		SMatOmegaMatProd = fjlt.gaussian_random_projection(SMat, EmbedRow)
	elif RndType == 'JLT':
		SMatOmegaMatProd = fjlt.fjlt(SMat, EmbedRow)
	else:
		sys.exit('Random Matrix type is neither JLT nor Gaussian')

	Q,R = LA.qr(SMatOmegaMatProd)
	MNumRows = SMatOmegaMatProd.shape[0]
	NNumCols = SMatOmegaMatProd.shape[1]
	QFinal = Q
	if MNumRows > NNumCols:
		QFinal = Q[:, np.arange(NNumCols)]

	return QFinal, NNumCols



In [7]:
def findATilde(QMat,AMat):
	Atilde = np.dot(np.dot(QMat.T, AMat), QMat)
	return Atilde


In [8]:
def findEMat(QMat, SMat):
	EMat = np.dot(np.dot(QMat.T, SMat), QMat)
	return EMat

In [9]:
def solveForPSDSymmetricP(EMat, AtildeMat, RNumCol):
	PMat = cp.Variable((RNumCol,RNumCol))
	objective_fn = cp.norm(EMat - cp.matmul(PMat,AtildeMat) - cp.matmul(AtildeMat, PMat),
							p='fro')
	prob = cp.Problem(cp.Minimize(objective_fn))

	prob.solve()
	PMatVal = PMat.value

	return PMatVal

In [10]:
def doCholeskyFactAbsEigenVal(PMat):
	print(PMat.shape)
	eigVal, eigVec = np.linalg.eig(PMat)
	eigValAbsDiagMat = np.diag(abs(eigVal))
	PMatPSD = np.dot(np.dot(eigVec, eigValAbsDiagMat), eigVec.T)
	UTildeMat = np.linalg.cholesky(PMatPSD)
	return UTildeMat


In [11]:
def findUMat(QMat, UTildeMat):
	UMat = np.dot(QMat, UTildeMat)
	return UMat

In [12]:
def checkResult(UMat, MStarMatDense, AMat):
	tempProd = MStarMatDense + np.dot(UMat, UMat.T)
	finalProd = np.dot(tempProd, AMat) + np.dot(AMat, tempProd)

In [13]:
def main():
	mean = np.zeros(64)
	CovMat = np.identity(64)
	AMat_dict = sio.loadmat("matrices/Trefethen_64.mat", squeeze_me=True)
	AMat = np.asarray(AMat_dict['tref2'].todense()) #FIXME: Change to sparse representation only
	SMat, MStarMatDense = findSMat("matrices/Trefethen_SSAI_64.mat", "Mst", AMat)
	QMat, RNumCols = findThinQ(SMat, 4, RndType='JLT')
	AMatTilde = findATilde(QMat, AMat)
	EMat = findEMat(QMat, SMat)

	PVal = solveForPSDSymmetricP(EMat,AMatTilde, RNumCols)
	UTildeMat = doCholeskyFactAbsEigenVal(PVal)
	UMat = findUMat(QMat, UTildeMat)
	checkResult(UMat, MStarMatDense, AMat)


In [15]:
if __name__ == '__main__':
  main()

FileNotFoundError: ignored